# Construindo uma CNN com Pytorch
- A intenção desse Notebook é aprendermos a construir uma CNN utilizando os building blocks do PyTorch
- Os conceitos são:
     - Carregar um dataset usando `torchvision.datasets`
     - Entender o `torch.utils.data.Dataset`
     - Entender o `torchvision.transform`
     - Aprender a usar um `Dataloader`
     - Criar um modelo estendendo a classe `nn.Module`
     - Treinar a rede em uma GPU 

In [2]:
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn

## Carregando a CIFAR10 novamente
- Para mais informações, veja o notebook da aula anterior

In [4]:
cifar_train = torchvision.datasets.CIFAR10(root="/home/patcha/datasets", 
                                          train=True, 
                                          download=True,
                                          transform=transforms.ToTensor())

cifar_test = torchvision.datasets.CIFAR10(root="/home/patcha/datasets", 
                                          train=False, 
                                          download=True,
                                          transform=transforms.ToTensor())

Files already downloaded and verified
Files already downloaded and verified


In [5]:
batch_size = 30
train_dataloader = torch.utils.data.DataLoader(dataset=cifar_train, 
                                               batch_size=batch_size, 
                                               shuffle=True)

test_dataloader = torch.utils.data.DataLoader(dataset=cifar_test, 
                                               batch_size=batch_size, 
                                               shuffle=True)

## Criando uma CNN
- Agora vamos criar nossa CNN
- Novamente, a maneira padrão de criar redes neurais dentro do Pytorch é estendendo a classe `nn.Module`
    - Lembra da ideia das peças de LEGO
- Neste caso, vamos usar as seguintes "peças":
    - `nn.Sequential`: https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html
    - `nn.Conv2d`: https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html
    - `nn.BatchNorm2d`: https://pytorch.org/docs/stable/generated/torch.nn.BatchNorm2d.html
    - `nn.ReLU`: https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html
    - `nn.MaxPool2d`: https://pytorch.org/docs/stable/generated/torch.nn.MaxPool2d.html

In [29]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.fc = nn.Linear(2048, num_classes)
        
    def forward(self, x):
        out = self.layer1(x)        
        out = self.layer2(out)
        # Fazendo a operação de flatten        
        out = out.reshape(out.size(0), -1)        
        out = self.fc(out)
        return out

- Agora podemos instanciar o nosso modelo:

In [30]:
model = ConvNet()
model

ConvNet(
  (layer1): Sequential(
    (0): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=2048, out_features=10, bias=True)
)

- Agora podemos determinar nossa função de custo e otimizador
- Para esse notebook vamos aproveitar o que já fizemos no anterior

In [31]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)  

- Agora vamos fazer nosso loop de treinamento
- Agora, vamos mandar nosso modelo para GPU se ela estiver disponível

In [32]:
num_epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Movendo o modelo para o device alvo
model.to(device)

for epoch in range(num_epochs):
    for k, (batch_images, batch_labels) in enumerate(train_dataloader):  
        
        # Aplicando um flatten na imagem e movendo ela para o device alvo
        batch_images = batch_images.to(device)
        batch_labels = batch_labels.to(device)
        
        # Fazendo a forward pass
        # observe que o modelo é agnóstico ao batch size
        outputs = model(batch_images)
        loss = loss_func(outputs, batch_labels)
        
        # Fazendo a otimização
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()        
    
    print (f"- Epoch [{epoch+1}/{num_epochs}] | Loss: {loss.item():.4f}")                  

- Epoch [1/10] | Loss: 1.5743
- Epoch [2/10] | Loss: 1.0791
- Epoch [3/10] | Loss: 0.9940
- Epoch [4/10] | Loss: 0.8071
- Epoch [5/10] | Loss: 0.9047
- Epoch [6/10] | Loss: 0.8011
- Epoch [7/10] | Loss: 0.8180
- Epoch [8/10] | Loss: 1.1310
- Epoch [9/10] | Loss: 0.8319
- Epoch [10/10] | Loss: 0.9865


### Fazendo inferência no conjunto de teste
- A inferência é basicamente igual a do notebook anterior

In [34]:
with torch.no_grad():
    correct, total = 0, 0
    for images, labels in test_dataloader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Accuracy: {100 * correct / total}%")

Accuracy: 68.66%


- Salvando o modelo:

In [ ]:
torch.save(model.state_dict(), 'model_conv.pth')

___
# Exercícios
- Estude a arquitetura da CNN desenvolvida nesse notebook e entenda o porque dos shapes de saída e entrada de cada camada
- Modifique a CNN para melhorarmos os resultado
___